In [1]:
import time
import pandas as pd
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore

In [2]:
cred = credentials.Certificate('../../src/config/smart-agri-bbd45-firebase-adminsdk-jfpri-5e03cc48f1.json')

app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
bulk_writer = db.bulk_writer()
batch = db.batch()
collection = db.collection('mrv_system')

In [4]:
location_data = ['latitude', 'longitude', 'district', 'province', 'address']
weather_data = ['max_temperature', 'min_temperature', 'wind', 'wind_direction', 'humidity', 'cloud',
                'atmospheric_pressure']
soil_data = ['soil_moisture', 'soil_moisture_20', 'soil_moisture_40', 'soil_temperature', 'soil_ph',
             'soil_conductivity']
irrigation_data = ['water_consumed', 'water_recycled', 'irrigation_type', 'days_flooded']
pest_data = ['pest_population_counts', 'disease_incidence', 'severity_of_infestations']
energy_data = ['diesel', 'gasoline', 'electricity']


In [5]:
def create_data_item(row):
    return {
        'fertilize': row.get('fertilize', None),
        'weather': {key: row.get(key, None) for key in weather_data},
        'pest': {key: row.get(key, None) for key in pest_data},
        'irrigation': {key: row.get(key, None) for key in irrigation_data},
        'soil': {key: row.get(key, None) for key in soil_data},
        'energy': {key: row.get(key, None) for key in energy_data}
    }

In [6]:
def write_data_to_firestore(df):
    bulk_writer = db.bulk_writer()

    for index, row in df.iterrows():
        print(f"index: {index}")
        data_item = create_data_item(row)
        facility = row.get('facility', None)
        doc_ref = db.collection("mrv_system").document()
        data = {
            'plant': str(row['plant']).lower().strip(),
            'date': datetime.strptime(
                datetime.strptime(row["datetime"].strftime("%d/%m/%Y"), "%d/%m/%Y").strftime("%Y/%m/%d"), "%Y/%m/%d"),
            'facility': str(facility).lower(),
            'location': {key: row.get(key, None) for key in location_data},
            'data': data_item
        }
        print(f"data: {data}")
        print("________")
        bulk_writer.set(doc_ref, data)

    bulk_writer.flush()

In [7]:
df = pd.read_excel('../data/data.xlsx', sheet_name='data_temp')

In [8]:
start = time.time()
write_data_to_firestore(df)
end = time.time()
print(f'Execution time: {end - start} seconds')

index: 0
data: {'plant': 'rice', 'date': datetime.datetime(2024, 3, 5, 0, 0), 'facility': 'dolphin', 'location': {'latitude': 20.5646621017849, 'longitude': 106.56213138823, 'district': 'Thai Thuy', 'province': 'Thai Binh', 'address': 'Bao Trình, thị trấn Diêm Điền, huyện Thái Thụy, tỉnh Thái Bình'}, 'data': {'fertilize': None, 'weather': {'max_temperature': 23.49, 'min_temperature': 19.04, 'wind': 9.63, 'wind_direction': 'N', 'humidity': 65.16, 'cloud': 77.71, 'atmospheric_pressure': 1002.02}, 'pest': {'pest_population_counts': 48, 'disease_incidence': 4.52, 'severity_of_infestations': 1.22}, 'irrigation': {'water_consumed': 887.23, 'water_recycled': 368.76, 'irrigation_type': 'continuous_flooding', 'days_flooded': 1}, 'soil': {'soil_moisture': 14.35, 'soil_moisture_20': 19.35, 'soil_moisture_40': 25.35, 'soil_temperature': 26.69, 'soil_ph': 6.93, 'soil_conductivity': 0.94}, 'energy': {'diesel': 0.0, 'gasoline': 0.0, 'electricity': 12.25}}}
________
index: 1
data: {'plant': 'rice', 'd